In [30]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from torch_geometric.data import Data
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score
from torch_geometric.data import Dataset

import clang.cindex
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

In [31]:
# Configure libclang path
if os.name == 'nt':  # Windows
    print('Windows')
    clang.cindex.Config.set_library_file('D:/Project/LLVM/bin/libclang.dll')
    print(clang.cindex.Config.library_path)
elif os.name == 'posix':  # Linux/Mac
    print('Linux/Mac')
    clang.cindex.Config.set_library_file('/Library/Developer/CommandLineTools/usr/lib/libclang.dylib')
    # clang.cindex.Config.set_library_path('/Library/Developer/CommandLineTools/usr/lib/')
    print(clang.cindex.Config.library_path)

# Verify if libclang is loaded
print(clang.cindex.Config.loaded)  # Should print `True`

Linux/Mac
None
False


In [32]:
def save_ast(node):
   
    node.children = list(node.get_children())

    for child in node.children:
        counter = save_ast(child)
        
def numbering_ast_nodes(node, counter=1):
  
    node.identifier = counter
    counter += 1

    node.children = list(node.get_children())
    for child in node.children:
        counter = numbering_ast_nodes(child, counter)

    return counter

def generate_edgelist(ast_root):
  
    edges = [[],[]]

    def walk_tree_and_add_edges(node):
        for child in node.children:
            # edges.append([node.identifier, child.identifier])
            # walk_tree_and_add_edges(child)
            edg_0 = (node.identifier)-1
            edg_1 = (child.identifier)-1
            # edges[0].append(node.identifier)
            # edges[1].append(child.identifier)
            edges[0].append(edg_0)
            edges[1].append(edg_1)
            walk_tree_and_add_edges(child)

    walk_tree_and_add_edges(ast_root)
    return  torch.tensor(edges, dtype=torch.long)

def generate_features(ast_root):
  
    features = []

    def walk_tree_and_set_features(node):
        out_degree = len(node.children)
        #in_degree = 1
        #degree = out_degree + in_degree
        degree = out_degree 
        node_id = node.identifier
        features.append([node_id, degree])

        for child in node.children:
            walk_tree_and_set_features(child)

    walk_tree_and_set_features(ast_root)

    features_array = np.asarray(features)
    # nodes_tensor = torch.from_numpy(features_array).float()
    nodes_tensor = torch.tensor(features_array, dtype=torch.float)
    # nodes_tensor = torch.LongTensor(features).unsqueeze(1)
    return nodes_tensor

def clang_process(testcase, **kwargs):
 
    parse_list = [
        (testcase.filename, testcase.code)
        
    ]

    # source_file= get_source_file(testcase)

    # Parsing the source code and extracting AST using clang
    index = clang.cindex.Index.create()
    translation_unit = index.parse(
        path=testcase.filename,
        unsaved_files=parse_list,
    )
    ast_root = translation_unit.cursor

    save_ast(ast_root)
    numbering_ast_nodes(ast_root)

    graphs_embedding = generate_edgelist(ast_root)

    nodes_embedding = generate_features(ast_root)
   

    y = torch.tensor([testcase.vuln], dtype=torch.int64)



    # delete clang objects
    del translation_unit
    del ast_root
    del index

    return Data(x=nodes_embedding, edge_index=graphs_embedding, y=y)

In [33]:
class MyOwnDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return 'not_implemented.pt'

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        self.data = pd.read_csv("Datasets/Normalized_CWE-469.csv")
        for index, vuln in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            data = clang_process(vuln)
            torch.save(data, os.path.join(self.processed_dir, f'data_{index}.pt'))

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))
        return data

In [ ]:
dataset = MyOwnDataset(root='Datasets/')

Processing...
100%|██████████| 5250/5250 [00:12<00:00, 410.48it/s]
Done!


In [ ]:
len(dataset)

5250

In [ ]:
print(dataset.num_features)

1
